# Importing datasets

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.' , one_hot = True , reshape = False)

import tensorflow as tf

learning_rate = 0.00001
epochs = 10
batch_size = 128

test_valid_size = 256

n_classes = 10
dropout = 0.75

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Weights and Bias

In [14]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([5, 5 , 1 , 32])),
    'wc2' : tf.Variable(tf.random_normal([5 , 5 , 32 , 64])),
    'wd1' : tf.Variable(tf.random_normal([7*7*64 , 1024])),
    'out' : tf.Variable(tf.random_normal([1024 , n_classes]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' :tf.Variable(tf.random_normal([n_classes]))
}

# Convolution and MaxPooling Layers

In [15]:
def conv2d(x , W , b , strides = 1):
    x = tf.nn.conv2d(x , W , strides = [1 , strides , strides , 1] , padding = 'SAME')
    x = tf.nn.bias_add(x , b)
    return tf.nn.relu(x)

def maxpool2d(x , k = 2):
    return tf.nn.max_pool(x , ksize = [1 , k , k , 1], strides = [1 , k , k , 1] , padding = 'SAME')

# Model

In [18]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Creating a session

In [ ]:
x = tf.placeholder(tf.float32 , [None , 28 , 28 , 1])
y = tf.placeholder(tf.float32 , [None , n_classes])
keep_prob = tf.placeholder(tf.float32)
logits = conv_net(x , weights , biases , keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits , labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(logits , 1), tf.argmax(y , 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x , batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer , feed_dict = {x:batch_x , y:batch_y , keep_prob : dropout})
            
            loss = sess.run(cost , feed_dict = {x:batch_x , y:batch_y , keep_prob : 1.})
            
            valid_acc = sess.run(accuracy , feed_dict = {x : mnist.validation.images[:test_valid_size] , y : mnist.validation.labels[:test_valid_size] , keep_prob : 1.})
            
            print('Epochs {:>2} , Batch {:>3} -' 'Loss {:>10.4f} Validation Accuracy: {:.6f}'.format(epochs + 1 , batch + 1 , loss , valid_acc))
    
    test_acc = sess.run(accuracy , feed_dict = {x : mnist.test.images[:test_valid_size], y : mnist.test.labels[:test_valid_size] , keep_prob : 1.})
    print('testing accuracy : {}'.format(test_acc))

Epochs 11 , Batch   1 -Loss 76687.2344 Validation Accuracy: 0.113281
Epochs 11 , Batch   2 -Loss 52650.4219 Validation Accuracy: 0.140625
Epochs 11 , Batch   3 -Loss 48002.8047 Validation Accuracy: 0.160156
Epochs 11 , Batch   4 -Loss 42168.2188 Validation Accuracy: 0.140625
Epochs 11 , Batch   5 -Loss 35007.4414 Validation Accuracy: 0.164062
Epochs 11 , Batch   6 -Loss 29292.9258 Validation Accuracy: 0.140625
Epochs 11 , Batch   7 -Loss 32944.3203 Validation Accuracy: 0.140625
Epochs 11 , Batch   8 -Loss 28276.4023 Validation Accuracy: 0.167969
Epochs 11 , Batch   9 -Loss 24721.0449 Validation Accuracy: 0.156250
Epochs 11 , Batch  10 -Loss 26736.8594 Validation Accuracy: 0.171875
Epochs 11 , Batch  11 -Loss 22485.4668 Validation Accuracy: 0.160156
Epochs 11 , Batch  12 -Loss 21681.3223 Validation Accuracy: 0.175781
Epochs 11 , Batch  13 -Loss 18006.7246 Validation Accuracy: 0.214844
Epochs 11 , Batch  14 -Loss 17782.7266 Validation Accuracy: 0.210938
Epochs 11 , Batch  15 -Loss 19377.

Epochs 11 , Batch 120 -Loss  2540.5950 Validation Accuracy: 0.589844
Epochs 11 , Batch 121 -Loss  3766.3481 Validation Accuracy: 0.593750
Epochs 11 , Batch 122 -Loss  4214.7861 Validation Accuracy: 0.593750
Epochs 11 , Batch 123 -Loss  3319.4841 Validation Accuracy: 0.589844
Epochs 11 , Batch 124 -Loss  4094.1780 Validation Accuracy: 0.585938
Epochs 11 , Batch 125 -Loss  3592.9009 Validation Accuracy: 0.605469
Epochs 11 , Batch 126 -Loss  3868.0889 Validation Accuracy: 0.609375
Epochs 11 , Batch 127 -Loss  2968.4470 Validation Accuracy: 0.605469
Epochs 11 , Batch 128 -Loss  3315.0054 Validation Accuracy: 0.609375
Epochs 11 , Batch 129 -Loss  3122.3999 Validation Accuracy: 0.613281
Epochs 11 , Batch 130 -Loss  3479.7983 Validation Accuracy: 0.609375
Epochs 11 , Batch 131 -Loss  3029.6838 Validation Accuracy: 0.613281
Epochs 11 , Batch 132 -Loss  2708.7456 Validation Accuracy: 0.613281
Epochs 11 , Batch 133 -Loss  3291.7312 Validation Accuracy: 0.613281
Epochs 11 , Batch 134 -Loss  4536.

Epochs 11 , Batch 239 -Loss  2676.2488 Validation Accuracy: 0.734375
Epochs 11 , Batch 240 -Loss  1658.8961 Validation Accuracy: 0.730469
Epochs 11 , Batch 241 -Loss  2869.3276 Validation Accuracy: 0.730469
Epochs 11 , Batch 242 -Loss  1701.9268 Validation Accuracy: 0.734375
Epochs 11 , Batch 243 -Loss  2055.2988 Validation Accuracy: 0.722656
Epochs 11 , Batch 244 -Loss  2155.3169 Validation Accuracy: 0.726562
Epochs 11 , Batch 245 -Loss  1842.3916 Validation Accuracy: 0.722656
Epochs 11 , Batch 246 -Loss  2651.2446 Validation Accuracy: 0.734375
Epochs 11 , Batch 247 -Loss  2269.0068 Validation Accuracy: 0.730469
Epochs 11 , Batch 248 -Loss  2728.9741 Validation Accuracy: 0.734375
Epochs 11 , Batch 249 -Loss  2589.6323 Validation Accuracy: 0.734375
Epochs 11 , Batch 250 -Loss  2540.3340 Validation Accuracy: 0.738281
Epochs 11 , Batch 251 -Loss  2284.5938 Validation Accuracy: 0.730469
Epochs 11 , Batch 252 -Loss  1450.6892 Validation Accuracy: 0.738281
Epochs 11 , Batch 253 -Loss  1910.

Epochs 11 , Batch 358 -Loss  1698.0686 Validation Accuracy: 0.765625
Epochs 11 , Batch 359 -Loss  1632.9753 Validation Accuracy: 0.773438
Epochs 11 , Batch 360 -Loss  1458.0652 Validation Accuracy: 0.777344
Epochs 11 , Batch 361 -Loss  1531.1829 Validation Accuracy: 0.757812
Epochs 11 , Batch 362 -Loss  1789.6962 Validation Accuracy: 0.761719
Epochs 11 , Batch 363 -Loss  2035.3748 Validation Accuracy: 0.777344
Epochs 11 , Batch 364 -Loss  1232.3818 Validation Accuracy: 0.761719
Epochs 11 , Batch 365 -Loss  1661.9764 Validation Accuracy: 0.769531
Epochs 11 , Batch 366 -Loss  1807.7384 Validation Accuracy: 0.777344
Epochs 11 , Batch 367 -Loss  1337.0580 Validation Accuracy: 0.777344
Epochs 11 , Batch 368 -Loss  1682.8486 Validation Accuracy: 0.781250
Epochs 11 , Batch 369 -Loss  1234.2035 Validation Accuracy: 0.785156
Epochs 11 , Batch 370 -Loss  1388.7610 Validation Accuracy: 0.781250
Epochs 11 , Batch 371 -Loss  1430.0554 Validation Accuracy: 0.785156
Epochs 11 , Batch 372 -Loss  1497.

Epochs 11 , Batch  48 -Loss  1240.6622 Validation Accuracy: 0.785156
Epochs 11 , Batch  49 -Loss   881.7088 Validation Accuracy: 0.785156
Epochs 11 , Batch  50 -Loss  1815.3054 Validation Accuracy: 0.789062
Epochs 11 , Batch  51 -Loss   566.7864 Validation Accuracy: 0.785156
Epochs 11 , Batch  52 -Loss  1698.9556 Validation Accuracy: 0.785156
Epochs 11 , Batch  53 -Loss   881.9620 Validation Accuracy: 0.789062
Epochs 11 , Batch  54 -Loss   882.0670 Validation Accuracy: 0.792969
Epochs 11 , Batch  55 -Loss  1341.6035 Validation Accuracy: 0.777344
Epochs 11 , Batch  56 -Loss  1214.2792 Validation Accuracy: 0.781250
Epochs 11 , Batch  57 -Loss  1353.0869 Validation Accuracy: 0.785156
Epochs 11 , Batch  58 -Loss  1490.2954 Validation Accuracy: 0.785156
Epochs 11 , Batch  59 -Loss   651.5015 Validation Accuracy: 0.785156
Epochs 11 , Batch  60 -Loss  1755.3723 Validation Accuracy: 0.789062
Epochs 11 , Batch  61 -Loss  1391.0896 Validation Accuracy: 0.789062
Epochs 11 , Batch  62 -Loss  1230.

...

Epochs 11 , Batch 402 -Loss   292.0425 Validation Accuracy: 0.855469
Epochs 11 , Batch 403 -Loss   385.4323 Validation Accuracy: 0.851562
Epochs 11 , Batch 404 -Loss   366.6771 Validation Accuracy: 0.851562
Epochs 11 , Batch 405 -Loss   345.7958 Validation Accuracy: 0.851562
Epochs 11 , Batch 406 -Loss   217.8456 Validation Accuracy: 0.851562
Epochs 11 , Batch 407 -Loss   377.5217 Validation Accuracy: 0.851562
Epochs 11 , Batch 408 -Loss   166.5810 Validation Accuracy: 0.855469
Epochs 11 , Batch 409 -Loss   160.5452 Validation Accuracy: 0.847656
Epochs 11 , Batch 410 -Loss   403.2029 Validation Accuracy: 0.847656
Epochs 11 , Batch 411 -Loss   282.8424 Validation Accuracy: 0.843750
Epochs 11 , Batch 412 -Loss   183.7352 Validation Accuracy: 0.847656
Epochs 11 , Batch 413 -Loss   212.4504 Validation Accuracy: 0.851562
Epochs 11 , Batch 414 -Loss   211.7621 Validation Accuracy: 0.851562
Epochs 11 , Batch 415 -Loss   163.9379 Validation Accuracy: 0.855469
Epochs 11 , Batch 416 -Loss   158.

Epochs 11 , Batch  92 -Loss   236.0872 Validation Accuracy: 0.855469
Epochs 11 , Batch  93 -Loss   251.9889 Validation Accuracy: 0.855469
Epochs 11 , Batch  94 -Loss   301.0636 Validation Accuracy: 0.855469
Epochs 11 , Batch  95 -Loss   413.0606 Validation Accuracy: 0.855469
Epochs 11 , Batch  96 -Loss   391.8008 Validation Accuracy: 0.851562
Epochs 11 , Batch  97 -Loss   304.5838 Validation Accuracy: 0.851562
Epochs 11 , Batch  98 -Loss   257.2264 Validation Accuracy: 0.851562
Epochs 11 , Batch  99 -Loss   281.8618 Validation Accuracy: 0.851562
Epochs 11 , Batch 100 -Loss   246.3677 Validation Accuracy: 0.847656
Epochs 11 , Batch 101 -Loss   487.5228 Validation Accuracy: 0.847656
Epochs 11 , Batch 102 -Loss   384.9926 Validation Accuracy: 0.851562
Epochs 11 , Batch 103 -Loss   470.2039 Validation Accuracy: 0.851562
Epochs 11 , Batch 104 -Loss   252.9857 Validation Accuracy: 0.851562
Epochs 11 , Batch 105 -Loss   174.2871 Validation Accuracy: 0.851562
Epochs 11 , Batch 106 -Loss   362.

Epochs 11 , Batch 211 -Loss   276.8490 Validation Accuracy: 0.843750
Epochs 11 , Batch 212 -Loss   164.6492 Validation Accuracy: 0.847656
Epochs 11 , Batch 213 -Loss   384.8628 Validation Accuracy: 0.847656
Epochs 11 , Batch 214 -Loss   262.1217 Validation Accuracy: 0.847656
Epochs 11 , Batch 215 -Loss   256.4502 Validation Accuracy: 0.843750
Epochs 11 , Batch 216 -Loss   468.1026 Validation Accuracy: 0.851562
Epochs 11 , Batch 217 -Loss   433.5733 Validation Accuracy: 0.851562
Epochs 11 , Batch 218 -Loss   359.4570 Validation Accuracy: 0.859375
Epochs 11 , Batch 219 -Loss   250.9125 Validation Accuracy: 0.859375
Epochs 11 , Batch 220 -Loss   355.5179 Validation Accuracy: 0.855469
Epochs 11 , Batch 221 -Loss   284.9069 Validation Accuracy: 0.859375
Epochs 11 , Batch 222 -Loss   374.5028 Validation Accuracy: 0.855469
Epochs 11 , Batch 223 -Loss   343.0026 Validation Accuracy: 0.855469
Epochs 11 , Batch 224 -Loss   189.7375 Validation Accuracy: 0.851562
Epochs 11 , Batch 225 -Loss   258.

Epochs 11 , Batch 330 -Loss   545.4777 Validation Accuracy: 0.855469
Epochs 11 , Batch 331 -Loss   241.8289 Validation Accuracy: 0.855469
Epochs 11 , Batch 332 -Loss   195.6442 Validation Accuracy: 0.859375
Epochs 11 , Batch 333 -Loss   267.5388 Validation Accuracy: 0.855469
Epochs 11 , Batch 334 -Loss   350.2686 Validation Accuracy: 0.855469
Epochs 11 , Batch 335 -Loss   303.2210 Validation Accuracy: 0.855469
Epochs 11 , Batch 336 -Loss   196.9868 Validation Accuracy: 0.859375
Epochs 11 , Batch 337 -Loss   207.6844 Validation Accuracy: 0.859375
Epochs 11 , Batch 338 -Loss   202.2420 Validation Accuracy: 0.859375
Epochs 11 , Batch 339 -Loss   192.2164 Validation Accuracy: 0.855469
Epochs 11 , Batch 340 -Loss   190.7977 Validation Accuracy: 0.859375
Epochs 11 , Batch 341 -Loss   426.5234 Validation Accuracy: 0.855469
Epochs 11 , Batch 342 -Loss   284.6204 Validation Accuracy: 0.855469
Epochs 11 , Batch 343 -Loss   150.5435 Validation Accuracy: 0.855469
Epochs 11 , Batch 344 -Loss   337.

Epochs 11 , Batch  20 -Loss   331.4560 Validation Accuracy: 0.851562
Epochs 11 , Batch  21 -Loss   397.2244 Validation Accuracy: 0.851562
Epochs 11 , Batch  22 -Loss   171.0523 Validation Accuracy: 0.851562
Epochs 11 , Batch  23 -Loss   188.7971 Validation Accuracy: 0.851562
Epochs 11 , Batch  24 -Loss   155.4778 Validation Accuracy: 0.851562
Epochs 11 , Batch  25 -Loss   304.6722 Validation Accuracy: 0.851562
Epochs 11 , Batch  26 -Loss   327.4700 Validation Accuracy: 0.851562
Epochs 11 , Batch  27 -Loss   241.3946 Validation Accuracy: 0.851562
Epochs 11 , Batch  28 -Loss   315.8971 Validation Accuracy: 0.851562
Epochs 11 , Batch  29 -Loss   208.0052 Validation Accuracy: 0.855469
Epochs 11 , Batch  30 -Loss   224.3528 Validation Accuracy: 0.855469
Epochs 11 , Batch  31 -Loss   160.9846 Validation Accuracy: 0.851562
Epochs 11 , Batch  32 -Loss   170.7478 Validation Accuracy: 0.851562
Epochs 11 , Batch  33 -Loss   229.1674 Validation Accuracy: 0.851562
Epochs 11 , Batch  34 -Loss   230.